In [521]:
import pandas as pd
import numpy as np

In [522]:
df = pd.read_csv('C:\\Users\\ACER\\OneDrive - Universitas Airlangga\\Kuliah\\Learn-MLZoomCamp2023\\Learn-MLZoomCamp2023\HomeWork\housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [523]:
df.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

In [524]:
df['ocean_proximity'].unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

In [525]:
record  = df[(df['ocean_proximity'] == '<1H OCEAN') | (df['ocean_proximity'] == 'INLAND')]

In [526]:
records = record.copy()

In [527]:
records['median_house_value'] = np.log1p(records['median_house_value'])

In [528]:
records.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [529]:
records.population.median()

1195.0

In [530]:
n = len(records)
n_val = int(0.2*n)
n_test = int(0.2*n)
n_train = n - (n_val + n_test)


n_test +n_train + n_val,n

(15687, 15687)

In [531]:
idx = np.arange(n)
np.random.seed(42)
np.random.shuffle(idx)
len(idx)

15687

In [532]:
del records['ocean_proximity']
df_train = records.iloc[idx[:n_train]]
df_val = records.iloc[idx[n_train : (n_train + n_val)]]
df_test = records.iloc[idx[(n_train + n_val):]]

df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)

y_train = df_train['median_house_value'].values
y_val = df_val['median_house_value'].values
y_test = df_test['median_house_value'].values

del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [533]:
def regresi_linier(x,y):
    ones = np.ones(x.shape[0])
    x = np.column_stack([ones,x])
    xtx = x.T.dot(x)
    xtx_inv = np.linalg.inv(xtx)
    w = xtx_inv.dot(x.T).dot(y)

    return w[0],w[1:]


In [534]:
def nol(train,val,test):
    df_train = train.copy()
    df_val = val.copy()
    df_test = test.copy()

    x_train = df_train.fillna(0).values
    x_val = df_val.fillna(0).values
    x_test = df_test.fillna(0).values
    #print((x_train == 0).sum())
    w0,wfull = regresi_linier(x_train,y_train)

    y_pred = w0 + x_val.dot(wfull)

    se = (y_pred - y_val)**2
    mse = se.mean()
    rmse = np.sqrt(mse)

    return rmse


In [535]:
def avg(train,val,test):
    df_train = train.copy()
    df_val = val.copy()
    df_test = test.copy()
    
    avg = df_train['total_bedrooms'].mean()
    
    x_train = df_train.fillna(avg).values
    x_val = df_val.fillna(avg).values
    x_test = df_test.fillna(avg).values
    
    print((x_train == 0).sum())
    w0,wfull = regresi_linier(x_train,y_train)

    y_pred = w0 + x_val.dot(wfull)
    
    se = (y_pred - y_val)**2
    mse = se.mean()
    rmse = np.sqrt(mse)

    return round(rmse,2)


In [536]:
print(f"RMSE fillna 0 = {nol(df_train,df_val,df_test)}")
print(f"RMSE fillna mean = {avg(df_train,df_val,df_test)}")

RMSE fillna 0 = 0.34084790341553023
0
RMSE fillna mean = 0.34


In [537]:
df_train['total_bedrooms'].mean()

542.552956325786

In [538]:
df_train.isnull().sum()

longitude              0
latitude               0
housing_median_age     0
total_rooms            0
total_bedrooms        94
population             0
households             0
median_income          0
dtype: int64

In [539]:
def regresi_linier_with_reg(x,y,r):
    ones = np.ones(x.shape[0])
    x = np.column_stack([ones,x])
    xtx = x.T.dot(x)
    xtx = xtx + r * np.eye(xtx.shape[0])
    
    xtx_inv = np.linalg.inv(xtx)
    w = xtx_inv.dot(x.T).dot(y)

    return w[0],w[1:]

In [540]:
def nol_r(train,val,test):
    df_train = train.copy()
    df_val = val.copy()
    df_test = test.copy()

    x_train = df_train.fillna(0).values
    x_val = df_val.fillna(0).values
    x_test = df_test.fillna(0).values

    reg = [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]
    for r in reg :
        w0,wfull = regresi_linier_with_reg(x_train,y_train,r)

        y_pred = w0 + x_val.dot(wfull)

        se = (y_pred - y_val)**2
        mse = se.mean()
        rmse = np.sqrt(mse)
        print(f"RMSE with r : {r} = {round(rmse,2)}")

In [541]:
nol_r(df_train,df_val,df_test)

RMSE with r : 0 = 0.34
RMSE with r : 1e-06 = 0.34
RMSE with r : 0.0001 = 0.34
RMSE with r : 0.001 = 0.34
RMSE with r : 0.01 = 0.34
RMSE with r : 0.1 = 0.34
RMSE with r : 1 = 0.34
RMSE with r : 5 = 0.35
RMSE with r : 10 = 0.35


In [544]:
rmse_list = []
for i in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    idx = np.arange(n)
    np.random.seed(i)
    np.random.shuffle(idx)

    df_train = records.iloc[idx[:n_train]].copy()
    df_val = records.iloc[idx[n_train : (n_train + n_val)]].copy()
    df_test = records.iloc[idx[(n_train + n_val):]].copy()

    df_train = df_train.reset_index(drop = True)
    df_test = df_test.reset_index(drop = True)
    df_val = df_val.reset_index(drop = True)

    y_train = df_train['median_house_value'].values
    y_val = df_val['median_house_value'].values
    y_test = df_test['median_house_value'].values

    del df_train['median_house_value']
    del df_val['median_house_value']
    del df_test['median_house_value']

    rmse = nol(df_train,df_val,df_test)
    rmse_list.append(rmse)
  
rmse_list

[0.3377387160088939,
 0.3377999353670445,
 0.3384287006767395,
 0.3320049468305144,
 0.339445186255509,
 0.34338197052682234,
 0.3385330211767963,
 0.3468747697296571,
 0.35127368659643043,
 0.3341558266512084]

In [545]:
np.round(np.std(rmse_list),3)

0.005

In [549]:
idx = np.arange(n)
np.random.seed(9)
np.random.shuffle(idx)

df_train = records.iloc[idx[:n_train]].copy()
df_val = records.iloc[idx[n_train : (n_train + n_val)]].copy()
df_test = records.iloc[idx[(n_train + n_val):]].copy()

df_full = pd.concat([df_train,df_val])
df_full = df_full.reset_index(drop = True)

df_test = df_test.reset_index(drop = True)

y_train = df_train['median_house_value'].values
y_val = df_val['median_house_value'].values
y_test = df_test['median_house_value'].values

y_full = np.concatenate([y_train,y_val])

del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']


x_train = df_train.fillna(0).values
x_val = df_val.fillna(0).values
x_test = df_test.fillna(0).values


w0,wfull = regresi_linier_with_reg(x_train,y_train,0.001)

y_pred = w0 + x_test.dot(wfull)

se = (y_pred - y_test)**2
mse = se.mean()
rmse = np.sqrt(mse)
print(f"RMSE = {round(rmse,2)}")

RMSE = 0.33
